# Explanaible AI - Shap
Documentation: 
- https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

## RUN

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

# shap
import shap

### 0. Global params

In [ ]:
# define folder where the models were saved. There are the same models accepted by gurobi but the feature eng changed

# list of folder with models = ['basic', 'scaler', 'poly_2', 'poly_3']
folder_models = 'basic'

### 1. Load data

In [ ]:
### DEFINE LIST FEARTURES - TARGET (order data to have the same order in the features always)
list_features = ['AveOccup', 'Latitude', 'Population', 'AveBedrms', 'HouseAge', 'Longitude', 'AveRooms', 'MedInc']
target = 'Price'

In [ ]:
### LOAD DATA
X_train = pd.read_pickle('artifacts/data/X_train.pkl')
X_test = pd.read_pickle('artifacts/data/X_test.pkl')
y_train = pd.read_pickle('artifacts/data/y_train.pkl')
y_test = pd.read_pickle('artifacts/data/y_test.pkl')

In [ ]:
print('shape data')
print('\n\n TRAIN')
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

print('\n\n TEST')
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

In [ ]:
# sample x train to run fast codes
X_train = X_train[0:1000]
X_train.shape

### 2. Load Models
Load all the models in a dictory

In [ ]:
## define list of models - list to have always the same order.
#### In this example, the strings in the list are the same with the models were saved
list_models_names = [
    "lr",
    "ridge",
    "lasso",
    
    "tree_simple",
    "tree_default",
    
    "rf_simple",
    "rf_default",

    "gb_simple",
    "gb_default",

    "xgb_simple",
    "xgb_default",

    "mlp_simple",
    "mlp_default"
]

In [ ]:
# define path to folder models
path_folder_models = f'artifacts/models/{folder_models}/'

In [ ]:
### load models
dict_models = {}
for model_name in list_models_names:
    print(f'loading model: {model_name}')
    path_model = path_folder_models + f'{model_name}.pkl'
    with open(path_model, 'rb') as artifact:
        dict_models[model_name] = pickle.load(artifact)

In [ ]:
# select model example
model = dict_models['lr']

### 3. Sample data

In [ ]:
# sample - run fast
X_train_100 = shap.utils.sample(X_train, 100)
y_train_100 = shap.utils.sample(y_train, 100)

In [ ]:
X_train.shape

In [ ]:
X_train_100.shape

### 4. Partial Dependence Plot
**Main**

To understand a feature’s importance in a model, it is necessary to understand both how changing that feature impacts the model’s output, and also the distribution of that feature’s values.


**Explication plot**

- Show the distribution of feature values as a histogram on the x-axis
- The gray horizontal line in the plot above represents the expected value of the model
- The vertical gray line represents the average value of the feature
- Note that the blue partial dependence plot line (which is the average value of the model output when we fix the median income feature to a given value) always passes through the intersection of the two gray expected value lines. We can consider this intersection point as the “center” of the partial dependence plot with respect to the data distribution

**Documentation**

- https://shap-lrjball.readthedocs.io/en/latest/generated/shap.partial_dependence_plot.html

In [ ]:
# select a feature
feature = "MedInc"

In [ ]:
shap.partial_dependence_plot(
    feature,
    model.predict,
    X_train_100,
    ice=False,
    model_expected_value=True,
    feature_expected_value=True,
)

**Aditional**
When we are explaining a prediction
, the SHAP value for a specific feature
 is just the difference between the expected model output and the partial dependence plot at the feature’s value 

In [ ]:
# compute the SHAP values for the linear model
explainer = shap.Explainer(model.predict, X_train_100)
shap_values = explainer(X_train)
shap_values

In [ ]:
# make a standard partial dependence plot
sample_ind = 20

In [ ]:
# TRY TO SOLVE PROBLEMS
# reduce dimensions of .base_values. # REDUCE ONE DIMENTION from (1,1) to (1,)
shap_values_to_partial_plot = shap_values[sample_ind : sample_ind + 1, :]
shap_values_to_partial_plot.base_values = shap_values_to_partial_plot.base_values[0]  # from (1,1) to (1,)
shap_values_to_partial_plot

In [ ]:
# partial dependence plot with shap values
shap.partial_dependence_plot(
    feature,
    model.predict,
    X_train_100,
    ice=False,
    model_expected_value=True,
    feature_expected_value=True,
    shap_values = shap_values[sample_ind : sample_ind + 1, :]  # plot a shap value
)

In [ ]:
# why this the code is failing
shap_values[sample_ind : sample_ind + 1, :]

In [ ]:
shap_values[sample_ind : sample_ind + 1, :].values.shape

In [ ]:
shap_values[sample_ind : sample_ind + 1, :].base_values.shape

In [ ]:
shap_values[sample_ind : sample_ind + 1, :].base_values

In [ ]:
shap_values[sample_ind : sample_ind + 1, :].data.shape

In [ ]:
shap.plots.scatter(shap_values[:, feature])

In [ ]:
shap_values#[:, feature]

In [ ]:
shap_values.values

In [ ]:
shap_values.values[:, feature]

In [ ]:
shap.plots.scatter(shap_values[:, "MedInc"])

# TODO: terminar de acuerdo a mi estudio de explanaible AI - termninar día viernes en la tarde. Migrar códigos viernes noche